In [2]:
import numpy as np
import pdb

def f(x):
    return x**3+2*x+3

class Remez():
    def __init__(self,n = 2, fun_f = None, x_list_comb = []):
        self.n = n # 多项式次数
        self.fun_f = fun_f # 给定函数
        self.x_list_comb = x_list_comb # 离散点集合
        self.f_list_comb = [self.fun_f(x) for x in self.x_list_comb]

        self.g_list = None
        self.epsilon = None
        self.g = None
        self.param_list = None # g参数集合
        self.e = 1e-12 # 收敛参数
        self.__random_choice()

    def __random_choice(self):
        self.choice = np.random.choice(len(self.x_list_comb), self.n +2,replace = False)
        self.choice = list(self.choice) # 标记选点
        self.choice.sort() # 排序
        self.x_list = [self.x_list_comb[i] for i in self.choice]
        self.f_list = [self.fun_f(x) for x in self.x_list]
        return
    
    def __minus(self):
        self.g_list = [float(self.g(x)) for x in self.x_list_comb]
        diff = [np.abs(self.f_list_comb[i] - self.g_list[i]) for i in range(len(self.x_list_comb))]
        #print(self.g_list)
        return diff # 全部点计算

    def __fit(self):
        tmp_M = []
        for i in range(self.n+2):
            param = [(-1)**i] # epsilon 系数
            param.extend([(self.x_list[i])**j for j in range(self.n+1)]) # g系数
            tmp_M.append(param)
        tmp_M = np.matrix(tmp_M)
        #print(tmp_M)
        tmp = np.dot(tmp_M.I,np.matrix(self.f_list).T)
        self.epsilon = tmp[0]
        self.param_list = tmp[1:]
        self.g = (lambda x: np.dot([x**i for i in range(self.n+1)],self.param_list)) # g 多项式函数
        return
    
    def main(self):
        epsilon_last = 10_000
        self.__fit()
        while np.abs(self.epsilon - epsilon_last)>self.e:
            epsilon_last = self.epsilon
            diff = self.__minus()
            flag = [(1 if i > self.epsilon else 0) for i in diff]
            if sum(flag) == 0 :
                break
            else: # 重新选点
                idx = flag.index(1)
                diff_idx = [np.abs(x-idx) for x in self.choice]
                replace_idx = diff_idx.index(min(diff_idx))
                self.choice[replace_idx] = idx
                self.choice.sort() # 排序
                self.x_list = [self.x_list_comb[i] for i in self.choice]
                self.f_list = [self.fun_f(x) for x in self.x_list]
                self.__fit()
        return self.param_list


In [3]:
a = Remez(n=2,fun_f=f,x_list_comb=[2,3,4,5,6,7])
a.main()

matrix([[ 70.5],
        [-53.5],
        [ 13.5]])